In [ ]:
# If we're running on Colab, install libraries

# TODO: When Colab can install gala, switch from astro-gala

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery astro-gala

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 9.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


## Working with Units

The measurements we will work with are physical quantities, which means that they have two parts, a value and a unit. For example, the coordinate 30∘
 has value 30 and its units are degrees.

Until recently, most scientific computation was done with values only; units were left out of the program altogether, often with catastrophic results.

Astropy provides tools for including units explicitly in computations, which makes it possible to detect errors before they cause disasters.

In [ ]:
import astropy.units as u

#u is an object that contains most common units and all SI units.

In [ ]:
#You can use dir to list them

dir(u)

['A',
 'AA',
 'AB',
 'ABflux',
 'ABmag',
 'AU',
 'Angstrom',
 'B',
 'Ba',
 'Barye',
 'Bi',
 'Biot',
 'Bol',
 'Bq',
 'C',
 'Celsius',
 'Ci',
 'CompositeUnit',
 'D',
 'DN',
 'Da',
 'Dalton',
 'Debye',
 'Decibel',
 'DecibelUnit',
 'Dex',
 'DexUnit',
 'EA',
 'EAU',
 'EB',
 'EBa',
 'EC',
 'ED',
 'EF',
 'EG',
 'EGal',
 'EH',
 'EHz',
 'EJ',
 'EJy',
 'EK',
 'EL',
 'EN',
 'EOe',
 'EOhm',
 'EP',
 'EPa',
 'ER',
 'ERy',
 'ES',
 'ESt',
 'ET',
 'EV',
 'EW',
 'EWb',
 'Ea',
 'Eadu',
 'Earcmin',
 'Earcsec',
 'Eau',
 'Eb',
 'Ebarn',
 'Ebeam',
 'Ebin',
 'Ebit',
 'Ebyte',
 'Ecd',
 'Echan',
 'Ecount',
 'Ect',
 'Ed',
 'Edeg',
 'Edyn',
 'EeV',
 'Eerg',
 'Eg',
 'Eh',
 'EiB',
 'Eib',
 'Eibit',
 'Eibyte',
 'Ek',
 'El',
 'Elm',
 'Elx',
 'Elyr',
 'Em',
 'Emag',
 'Emin',
 'Emol',
 'Eohm',
 'Epc',
 'Eph',
 'Ephoton',
 'Epix',
 'Epixel',
 'Equivalency',
 'Erad',
 'Es',
 'Esr',
 'Eu',
 'Evox',
 'Evoxel',
 'Eyr',
 'F',
 'Farad',
 'Fr',
 'Franklin',
 'FunctionQuantity',
 'FunctionUnitBase',
 'G',
 'GA',
 'GAU',
 'GB',


In [ ]:
#To create a quantity, we multiply a value by a unit.
angle = 10 * u.degree
type(angle)



astropy.units.quantity.Quantity

In [ ]:
##The result is a Quantity object. Jupyter knows how to display Quantities like this:
angle

<Quantity 10. deg>

In [ ]:
#Quantities provide a method called to that converts to other units. For example, we can compute the number of arcminutes in angle:

angle_arcmin = angle.to(u.arcmin)
angle_arcmin

<Quantity 600. arcmin>

In [ ]:
#If you add quantities, Astropy converts them to compatible units, if possible:
angle + 30 * u.arcmin

<Quantity 10.5 deg>

*If the units are not compatible, you get an error. For example:*

->angle + 5 * u.second

causes a UnitConversionError

***Let's Create a quantity that represents 5 arcminutes and assign it to a variable called radius***

In [ ]:
radius = 5 * u.arcmin
radius

<Quantity 5. arcmin>

**Selecting a Region**

One of the most common ways to restrict a query is to select stars in a particular region of the sky.

In [ ]:
#here's a query that selects objects in a circular region centered at (88.8, 7.4) with a search radius of 5 arcmin (0.08333 deg).

query_cone = """SELECT
TOP 10
source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""

In [ ]:
!pip install astroquery

  Using cached astroquery-0.4.7-py3-none-any.whl.metadata (7.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.8/994.8 kB 37.3 MB/s eta 0:00:00


A query like this is called a cone search because it selects stars in a cone. Here's how we run it.

In [ ]:
from astroquery.gaia import Gaia

job = Gaia.launch_job(query_cone)
job

In [ ]:
results = job.get_results()
results

SOURCE_ID
int64
3322773965056065536
3322773758899157120
3322774068134271104
3322773930696320512
3322774377374425728
3322773724537891456
3322773724537891328
3322773930696321792
3322773724537890944


When you are debugging queries like this, you can use TOP to limit the size of the results, but then you still don't know how big the results will be.

An alternative is to use COUNT, which asks for the number of rows that would be selected, but it does not return them.

From the previous query, we will replace TOP 10 source_id with COUNT(source_id) and run the query again. How many stars has Gaia identified in the cone we searched?

In [ ]:
query_cone = """SELECT
count(source_id)
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""

In [ ]:
job2 = Gaia.launch_job(query_cone)
job2

In [ ]:
results2 = job2.get_results()
results2

COUNT
int64
594


## Getting GD-1 Data
From the Price-Whelan and Bonaca paper, we will try to reproduce Figure 1, which includes this representation of stars likely to belong to GD-1.

(GD-1 is a stellar stream and is located in the halo of the Milky Way, far from the galactic disk. It stretches across a large portion of the sky, making it one of the most well-known and studied stellar streams in the Milky Way.)

![Description of image](https://raw.githubusercontent.com/datacarpentry/astronomy-python/gh-pages/fig/gd1-4.png)

Transforming coordinates
Astropy provides a SkyCoord object that represents sky coordinates relative to a specified frame.

The following example creates a SkyCoord object that represents the approximate coordinates of Betelgeuse (alf Ori) in the ICRS frame.

ICRS is the "International Celestial Reference System", adopted in 1997 by the International Astronomical Union.

In [23]:
from astropy.coordinates import SkyCoord

ra = 88.8 * u.degree
dec = 7.4 * u.degree
coord_icrs = SkyCoord(ra=ra, dec=dec, frame='icrs')

coord_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    (88.8, 7.4)>

In [24]:
#SkyCoord provides a function that transforms to other frames. For example, we can transform coords_icrs to Galactic coordinates like this:
coord_galactic = coord_icrs.transform_to('galactic')
coord_galactic

<SkyCoord (Galactic): (l, b) in deg
    (199.79693102, -8.95591653)>

Notice that in the Galactic frame, the coordinates are called l and b, not ra and dec.

To transform to and from GD-1 coordinates, we'll use a frame defined by Gala, which is an Astropy-affiliated library that provides tools for galactic dynamics.

Gala provides GD1Koposov10, which is "a Heliocentric spherical coordinate system defined by the orbit of the GD-1 stream".

In [28]:
!python -m pip install gala

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 64.8 MB/s eta 0:00:00


In [30]:
from gala.coordinates import GD1Koposov10

gd1_frame = GD1Koposov10()
gd1_frame

<GD1Koposov10 Frame>

In [31]:
# We can use it to find the coordinates of Betelgeuse in the GD-1 frame, like this:
coord_gd1 = coord_icrs.transform_to(gd1_frame)
coord_gd1

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    (-94.97222038, 34.5813813)>

The coordinates are called phi1 and phi2. These are the coordinates shown in the figure from the paper, above.

Let's find the location of GD-1 in ICRS coordinates.

Create a SkyCoord object at 0°, 0° in the GD-1 frame.

Transform it to the ICRS frame.

Because ICRS is built into Astropy, you can specify it by name, icrs (as we did with galactic).

In [39]:
from astropy.coordinates import SkyCoord
from astropy import units as u

# Step 1: Create a SkyCoord object at 0°, 0° in ICRS frame (RA=0, Dec=0)
ra = 0.0 * u.degree
dec = 0.0 * u.degree
coord_icrs2 = SkyCoord(ra=ra, dec=dec, frame='icrs')

# Step 2: Print the ICRS coordinates
print(coord_icrs2)

<SkyCoord (ICRS): (ra, dec) in deg
    (0., 0.)>


In [40]:
# Step 3: Transform the coordinates from ICRS to GD-1 (if GD-1 frame is predefined)
# Assuming GD-1 frame is defined, you would do:
coord_gd2 = coord_icrs2.transform_to(gd1_frame)

# Step 4: Print the GD-1 coordinates
print(coord_gd2)

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    (133.07549757, 45.62498003)>


## Selecting a rectangle
Now we'll use these coordinate transformations to define a rectangle in the GD-1 frame and transform it to ICRS.

The following variables define the boundaries of the rectangle in ϕ1 and ϕ2

In [41]:
phi1_min = -55 * u.degree
phi1_max = -45 * u.degree
phi2_min = -8 * u.degree
phi2_max = 4 * u.degree

In [42]:
# To create a rectangle, we'll use the following function, which takes the lower and upper bounds as parameters.
def make_rectangle(x1, x2, y1, y2):
    """Return the corners of a rectangle."""
    xs = [x1, x1, x2, x2, x1]
    ys = [y1, y2, y2, y1, y1]
    return xs, ys

In [43]:
# The return value is a tuple containing a list of coordinates in phi1 followed by a list of coordinates in phi2.

phi1_rect, phi2_rect = make_rectangle(
    phi1_min, phi1_max, phi2_min, phi2_max) #phi1_rect and phi2_rect contains the coordinates of the corners of a rectangle in the GD-1 frame.

In [44]:
# In order to use them in a Gaia query, we have to convert them to ICRS. First we'll put them into a SkyCoord object.
corners = SkyCoord(phi1=phi1_rect, phi2=phi2_rect, frame=gd1_frame)
corners

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    [(-55., -8.), (-55.,  4.), (-45.,  4.), (-45., -8.), (-55., -8.)]>

In [45]:
# Now we can use transform_to to convert to ICRS coordinates.
corners_icrs = corners.transform_to('icrs')
corners_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    [(146.27533314, 19.26190982), (135.42163944, 25.87738723),
     (141.60264825, 34.3048303 ), (152.81671045, 27.13611254),
     (146.27533314, 19.26190982)]>

## Defining a polygon
In order to use this polygon as part of an ADQL query, we have to convert it to a string with a comma-separated list of coordinates, as in this example:

"""
POLYGON(143.65, 20.98,
        134.46, 26.39,
        140.58, 34.85,
        150.16, 29.01)
"""
SkyCoord provides to_string, which produces a list of strings.

In [46]:
t = corners_icrs.to_string()
t

['146.275 19.2619',
 '135.422 25.8774',
 '141.603 34.3048',
 '152.817 27.1361',
 '146.275 19.2619']

In [47]:
# We can use the Python string function join to join t into a single string (with spaces between the pairs):
s = ' '.join(t)
s


'146.275 19.2619 135.422 25.8774 141.603 34.3048 152.817 27.1361 146.275 19.2619'

In [48]:
# That's almost what we need, but we have to replace the spaces with commas
s.replace(' ', ', ')

'146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619'

In [49]:
# Let's create a fnction to combines these steps.
def skycoord_to_string(skycoord):
    """Convert SkyCoord to string."""
    t = skycoord.to_string()
    s = ' '.join(t)
    return s.replace(' ', ', ')

In [50]:
point_list = skycoord_to_string(corners_icrs)
point_list

'146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619'

## Assembling the query
Now we're ready to assemble the query. We need columns again (as we saw in the previous lesson).

In [51]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax'

In [52]:
query3_base = """SELECT
TOP 10
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2
"""

In [53]:
# we'll add a WHERE clause to select stars in the polygon we defined.

query4_base = """SELECT
TOP 10
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2
  AND 1 = CONTAINS(POINT(ra, dec),
                   POLYGON({point_list}))
"""

In [54]:
#The query base contains format specifiers for columns and point_list.
#We'll use format to fill in these values.

query4 = query4_base.format(columns=columns,
                          point_list=point_list)
print(query4)

SELECT
TOP 10
source_id, ra, dec, pmra, pmdec, parallax
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON(146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619))



In [55]:
# As always, we should take a minute to proof-read the query before we launch it.
job = Gaia.launch_job_async(query4)
print(job)

INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
<Table length=10>
   name    dtype    unit                              description                            
--------- ------- -------- ------------------------------------------------------------------
SOURCE_ID   int64          Unique source identifier (unique within a particular Data Release)
       ra float64      deg                                                    Right ascension
      dec float64      deg                                                        Declination
     pmra float64 mas / yr                         Proper motion in right ascension direction
    pmdec float64 mas / yr                             Proper motion in declination direction
 parallax float64      mas                                                           Parallax
Jobid: 1734941698595O
Phase: COMPLETED
Owner: None
Output file: async_20241223081458.vot
Results: None


In [56]:
results = job.get_results()
results

SOURCE_ID,ra,dec,pmra,pmdec,parallax
,deg,deg,mas / yr,mas / yr,mas
int64,float64,float64,float64,float64,float64
637987125186749568,142.48301935991023,21.75771616932985,-2.5168384683875766,2.941813096629439,-0.2573448962333354
638285195917112960,142.25452941346344,22.476168171141378,2.6627020143457996,-12.165984395577347,0.4227283465319491
638073505568978688,142.64528557468074,22.16693224953078,18.30674739454163,-7.950659620550862,0.10363972229362585
638086386175786752,142.57739430926034,22.22791951401365,0.9877856720147953,-2.584105480335548,-0.8573270355079308
638049655615392384,142.58913564478618,22.110783166677418,0.24443878227817095,-4.941079187010136,0.099624729200593
638267565075964032,141.81762228999614,22.375696125322275,-3.413174589660796,1.8838892877285924,-0.07271215219283075
638028902333511168,143.18339801317677,22.2512465812369,7.848511762712128,-21.391145547787154,0.28736226011479443
638085767700610432,142.9347319464589,22.46244080823965,-3.6585960944321476,-12.486419770278376,-0.9896728393047383


In [57]:
#Finally, we can remove TOP 10 run the query again.
#The result is bigger than our previous queries, so it will take a little longer.

query5_base = """SELECT
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2
  AND 1 = CONTAINS(POINT(ra, dec),
                   POLYGON({point_list}))
"""

In [58]:
query5 = query5_base.format(columns=columns,
                          point_list=point_list)
print(query5)

SELECT
source_id, ra, dec, pmra, pmdec, parallax
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON(146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619))



In [59]:
job = Gaia.launch_job_async(query5)
print(job)

INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
<Table length=140339>
   name    dtype    unit                              description                            
--------- ------- -------- ------------------------------------------------------------------
SOURCE_ID   int64          Unique source identifier (unique within a particular Data Release)
       ra float64      deg                                                    Right ascension
      dec float64      deg                                                        Declination
     pmra float64 mas / yr                         Proper motion in right ascension direction
    pmdec float64 mas / yr                             Proper motion in declination direction
 parallax float64      mas                                                           Parallax
Jobid: 1734941766823O
Phase: COMPLETED
Owner: None
Output file: async_20241223081606.vot
Results: None


In [60]:
results = job.get_results()
len(results)

140339

*There are more than 100,000 stars in this polygon, but that's a manageable size to work with.*

## Saving results
This is the set of stars we'll work with in the next step. But since we have a substantial dataset now, this is a good time to save it.

Storing the data in a file means we can shut down this notebook and pick up where we left off without running the previous query again.

Astropy Table objects provide write, which writes the table to disk.

In [66]:
filename = 'gd1_results.fits'
results.write(filename, overwrite=True)

In [63]:
from os.path import getsize

MB = 1024 * 1024
getsize(filename) / MB

6.4324951171875

In [68]:
!pip install ipyaladin ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 33.4 MB/s eta 0:00:00


In [69]:
import ipyaladin
from ipywidgets import VBox

# Path to your FITS file
fits_file_path = '/content/gd1_results.fits'  # Replace with the path to your FITS file

# Create an Aladin viewer instance
aladin = ipyaladin.Aladin(fits=fits_file_path)

# Display the viewer in Jupyter Notebook
VBox([aladin])
